In [1]:
#loading my libraries - commenting out what I won't find useful for this excercise

import nltk
import numpy as np
import pandas as pd
from nltk.corpus import stopwords

from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from sklearn.model_selection import train_test_splitb

#from sklearn.model_selection import cross_val_score, cross_val_predict, GridSearchCV

from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.linear_model import SGDClassifier
# from sklearn.pipeline import Pipeline
#from sklearn.metrics import confusion_matrix, accuracy_score

In [2]:
data = pd.read_csv("../../data/final_data.csv")

In [4]:
data = data.drop(columns = ["Unnamed: 0"])

In [5]:
data

,provided_drug_name,condition,review,rating,date,usefulCount,matched_generic_name,matched_type
0,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5,14-Dec-09,17,NaN,NaN
1,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8,3-Nov-15,10,NaN,NaN
2,Levonorgestrel,Emergency Contraception,"""He pulled out, but he cummed a bit in me. I t...",1,7-Mar-17,5,levonorgestrel,emergency
3,Ethinyl estradiol / levonorgestrel,Birth Control,"""I had been on the pill for many years. When m...",8,8-Dec-16,1,ethinyl estradiol / norethindrone,standard
4,Nexplanon,Birth Control,"""Started Nexplanon 2 months ago because I have...",3,7-Aug-14,10,etonogestrel,standard
...,...,...,...,...,...,...,...,...
31246,Portia,Birth Control,"""I switched to Portia 12 days ago when I start...",1,28-Jan-14,9,ethinyl estradiol / levonorgestrel,standard
31247,Ethinyl estradiol / norethindrone,Birth Control,"""When I first starting taking Lo Loestrin Fe, ...",10,20-Apr-15,13,ethinyl estradiol / norethindrone,standard
31248,Mirena,Birth Control,"""My experience: Painful insertion but I expect...",10,26-Jan-16,4,levonorgestrel,standard
31249,Nexplanon,Birth Control,"""I have had the Nexplanon since Dec. 27, 2016 ...",6,6-Apr-17,0,etonogestrel,standard


In [ ]:
#double checking if there are any nulls in other columns than "matched_generic_name" or	"matched_type"

In [6]:
data.isnull().sum()

provided_drug_name         0
condition                  0
review                     0
rating                     0
date                       0
usefulCount                0
matched_generic_name    1840
matched_type            1840
dtype: int64

In [9]:
pd.set_option('display.max_colwidth', None)

In [10]:
data["review"][:10]

0                                   "I used to take another oral contraceptive, which had 21 pill cycle, and was very happy- very light periods, max 5 days, no other side effects. But it contained hormone gestodene, which is not available in US, so I switched to Lybrel, because the ingredients are similar. When my other pills ended, I started Lybrel immediately, on my first day of period, as the instructions said. And the period lasted for two weeks. When taking the second pack- same two weeks. And now, with third pack things got even worse- my third period lasted for two weeks and now it&#039;s the end of the third week- I still have daily brown discharge.\r\nThe positive side is that I didn&#039;t have any other side effects. The idea of being period free was so tempting... Alas."
1                                                                                                                                                                                                             

In [ ]:
#Now, I'll split reviews into negative and positive. Negative reviews have rating 1-5 (inclusive), positive: 6-10.

In [18]:
data.loc[(data["rating"] >= 1) & (data["rating"] <= 5), "sentiment"] = "negative"
data.loc[(data["rating"] >= 6) & (data["rating"] <= 10), "sentiment"] = "positive"

In [22]:
#for my future modelling, I should actually replace "negative", "positive" clasification into a binary one:
# 0 for negative, 1 for positive
#let's create another column "binary_target" based on those rules
# it depends on the model - some models complain 

data.loc[data["sentiment"] == "negative", "binary_target"] = int(0)
data.loc[data["sentiment"] == "positive", "binary_target"] = int(1)

In [23]:
data[["review", "rating", "sentiment", "binary_target"]]

,review,rating,sentiment,binary_target
0,"""I used to take another oral contraceptive, which had 21 pill cycle, and was very happy- very light periods, max 5 days, no other side effects. But it contained hormone gestodene, which is not available in US, so I switched to Lybrel, because the ingredients are similar. When my other pills ended, I started Lybrel immediately, on my first day of period, as the instructions said. And the period lasted for two weeks. When taking the second pack- same two weeks. And now, with third pack things got even worse- my third period lasted for two weeks and now it&#039;s the end of the third week- I still have daily brown discharge.\r\nThe positive side is that I didn&#039;t have any other side effects. The idea of being period free was so tempting... Alas.""",5,negative,0.0
1,"""This is my first time using any form of birth control. I&#039;m glad I went with the patch, I have been on it for 8 months. At first It decreased my libido but that subsided. The only downside is that it made my periods longer (5-6 days to be exact) I used to only have periods for 3-4 days max also made my cramps intense for the first two days of my period, I never had cramps before using birth control. Other than that in happy with the patch""",8,positive,1.0
2,"""He pulled out, but he cummed a bit in me. I took the Plan B 26 hours later, and took a pregnancy test two weeks later - - I&#039;m pregnant.""",1,negative,0.0
3,"""I had been on the pill for many years. When my doctor changed my RX to chateal, it was as effective. It really did help me by completely clearing my acne, this takes about 6 months though. I did not gain extra weight, or develop any emotional health issues. I stopped taking it bc I started using a more natural method of birth control, but started to take it bc I hate that my acne came back at age 28. I really hope symptoms like depression, or weight gain do not begin to affect me as I am older now. I&#039;m also naturally moody, so this may worsen things. I was in a negative mental rut today. Also I hope this doesn&#039;t push me over the edge, as I believe I am depressed. Hopefully it&#039;ll be just like when I was younger.""",8,positive,1.0
4,"""Started Nexplanon 2 months ago because I have a minimal amount of contraception&#039;s I can take due to my inability to take the hormone that is used in most birth controls. I&#039;m trying to give it time because it is one of my only options right now. But honestly if I had options I&#039;d get it removed.\r\nI&#039;ve never had acne problems in my life, and immediately broke out after getting it implanted. Sex drive is completely gone, and I used to have sex with my boyfriend a few days a week, now its completely forced and not even fun for me anymore. I mean I&#039;m on birth control because I like having sex but don&#039;t want to get pregnant, why take a birth control that takes away sex? Very unhappy and hope that I get it back with time or I&#039;m getting it removed.""",3,negative,0.0
...,...,...,...,...
31246,"""I switched to Portia 12 days ago when I started spotting from another birth control (Alesse). I just wanted to say I started taking Portia on Tuesday and have been crying for consecutive days from Friday to Tuesday out of no where. I have also found it very hard to get studying and made me a much lazier person. I will be discontinuing this birth control today because my mood swings were affecting me too much.""",1,negative,0.0
31247,"""When I first starting taking Lo Loestrin Fe, it was the first birth control pill I had taken, so I was expecting to become a little moody (which had subsided after about two weeks), but besides that I was completely fine. The first month on the pill I had a little breakthrough bleeding (very little brown discharge-- only for a couple of days), and this too had subsided after the first month. Ever since then I have had no period, no weight gain, no cramps, no breakthrough bleeding, no spotting, no nothing!

In [26]:
data["binary_target"] = data["binary_target"].astype(int)

In [27]:
data[["review", "rating", "sentiment", "binary_target"]]

,review,rating,sentiment,binary_target
0,"""I used to take another oral contraceptive, which had 21 pill cycle, and was very happy- very light periods, max 5 days, no other side effects. But it contained hormone gestodene, which is not available in US, so I switched to Lybrel, because the ingredients are similar. When my other pills ended, I started Lybrel immediately, on my first day of period, as the instructions said. And the period lasted for two weeks. When taking the second pack- same two weeks. And now, with third pack things got even worse- my third period lasted for two weeks and now it&#039;s the end of the third week- I still have daily brown discharge.\r\nThe positive side is that I didn&#039;t have any other side effects. The idea of being period free was so tempting... Alas.""",5,negative,0
1,"""This is my first time using any form of birth control. I&#039;m glad I went with the patch, I have been on it for 8 months. At first It decreased my libido but that subsided. The only downside is that it made my periods longer (5-6 days to be exact) I used to only have periods for 3-4 days max also made my cramps intense for the first two days of my period, I never had cramps before using birth control. Other than that in happy with the patch""",8,positive,1
2,"""He pulled out, but he cummed a bit in me. I took the Plan B 26 hours later, and took a pregnancy test two weeks later - - I&#039;m pregnant.""",1,negative,0
3,"""I had been on the pill for many years. When my doctor changed my RX to chateal, it was as effective. It really did help me by completely clearing my acne, this takes about 6 months though. I did not gain extra weight, or develop any emotional health issues. I stopped taking it bc I started using a more natural method of birth control, but started to take it bc I hate that my acne came back at age 28. I really hope symptoms like depression, or weight gain do not begin to affect me as I am older now. I&#039;m also naturally moody, so this may worsen things. I was in a negative mental rut today. Also I hope this doesn&#039;t push me over the edge, as I believe I am depressed. Hopefully it&#039;ll be just like when I was younger.""",8,positive,1
4,"""Started Nexplanon 2 months ago because I have a minimal amount of contraception&#039;s I can take due to my inability to take the hormone that is used in most birth controls. I&#039;m trying to give it time because it is one of my only options right now. But honestly if I had options I&#039;d get it removed.\r\nI&#039;ve never had acne problems in my life, and immediately broke out after getting it implanted. Sex drive is completely gone, and I used to have sex with my boyfriend a few days a week, now its completely forced and not even fun for me anymore. I mean I&#039;m on birth control because I like having sex but don&#039;t want to get pregnant, why take a birth control that takes away sex? Very unhappy and hope that I get it back with time or I&#039;m getting it removed.""",3,negative,0
...,...,...,...,...
31246,"""I switched to Portia 12 days ago when I started spotting from another birth control (Alesse). I just wanted to say I started taking Portia on Tuesday and have been crying for consecutive days from Friday to Tuesday out of no where. I have also found it very hard to get studying and made me a much lazier person. I will be discontinuing this birth control today because my mood swings were affecting me too much.""",1,negative,0
31247,"""When I first starting taking Lo Loestrin Fe, it was the first birth control pill I had taken, so I was expecting to become a little moody (which had subsided after about two weeks), but besides that I was completely fine. The first month on the pill I had a little breakthrough bleeding (very little brown discharge-- only for a couple of days), and this too had subsided after the first month. Ever since then I have had no period, no weight gain, no cramps, no breakthrough bleeding, no spotting, no nothing!""",10,posit

In [ ]:
#I have some weird symbols in my reviews, lik: "&#039;" - ASCII , " \r\r\nI" - tab - i should bear them in mind when removing (FOR LATER)


In [31]:
#Now, I want to check the distribution of my target var

target_distr = data["binary_target"].value_counts()
target_distr*100/len(data)

binary_target
1    59.655691
0    40.344309
Name: count, dtype: float64

In [34]:
#DO I NEED TO DROP THE SENTIMENT AS IN THE EXAMPLE FROM THE 2ND CLASS?

#sample = sample.drop('target', axis=1)  # you need to drop this column otherwise you'll be cheating the model

data = data.drop("sentiment", axis=1)

In [ ]:
#Splitting my data for training and test:
# - 80% for training and 20% for the test
# - I need a stratified sample (bc of 40-60% distribution)

In [36]:
X_train, X_test, y_train, y_test = \
train_test_split(data["review"], data["binary_target"], test_size=0.2, random_state=123) #random is reproducable; stratified is deafult?

In [44]:
print(f'Train dimensions: {X_train.shape, y_train.shape}')
print(f'Test dimensions: {X_test.shape, y_test.shape}' "\n")# Check out target distribution

print(f"Distribution of train binary_target variable for the train set: {y_train.value_counts()}", "\n")

print(f"Distribution of train binary_target variable for the test set: {y_test.value_counts()}")

Train dimensions: ((25000,), (25000,))
Test dimensions: ((6251,), (6251,))

Distribution of train binary_target variable for the train set: binary_target
1    14864
0    10136
Name: count, dtype: int64 

Distribution of train binary_target variable for the test set: binary_target
1    3779
0    2472
Name: count, dtype: int64


In [ ]:
#Preprocessing data 

In [51]:
eeview_for_processing = X_train[1]
review_for_processing

'"This is my first time using any form of birth control. I&#039;m glad I went with the patch, I have been on it for 8 months. At first It decreased my libido but that subsided. The only downside is that it made my periods longer (5-6 days to be exact) I used to only have periods for 3-4 days max also made my cramps intense for the first two days of my period, I never had cramps before using birth control. Other than that in happy with the patch"'

In [52]:
def preprocess_text(text):
    tokeniser = RegexpTokenizer(r'\w+') #I keep only words - same what Celeste did
    tokens = tokeniser.tokenize(text)

    #Lower case and lemmatise
    lemmatiser = WordNetLemmatizer()
    lemmas = [lemmatiser.lemmatize(token.lower(), pos="v") for token in tokens]
    lemmas = [lemmatiser.lemmatize(token.lower(), pos="n") for token in lemmas]

    #Remove stop words
    keywords = [lemma for lemma in lemmas if lemma not in stopwords.words("english")]
    return keywords

In [53]:
tokens = preprocess_text(text=review_for_processing)

In [54]:
tokens

['first',
 'time',
 'use',
 'form',
 'birth',
 'control',
 '039',
 'glad',
 'go',
 'patch',
 '8',
 'months',
 'first',
 'decrease',
 'libido',
 'subside',
 'downside',
 'make',
 'periods',
 'longer',
 '5',
 '6',
 'days',
 'exact',
 'use',
 'periods',
 '3',
 '4',
 'days',
 'max',
 'also',
 'make',
 'cramp',
 'intense',
 'first',
 'two',
 'days',
 'period',
 'never',
 'cramp',
 'use',
 'birth',
 'control',
 'happy',
 'patch']

In [ ]:
#preparing reviews for the model using tfidf

In [55]:
vectoriser = TfidfVectorizer(analyzer=preprocess_text) # this is the method you've implemented above
X_train_tfidf = vectoriser.fit_transform(X_train)  # notice that you are doing the fit only for train data

In [56]:
X_train_tfidf.shape

(25000, 11432)

In [57]:
X_test_tfidf = vectoriser.transform(X_test) #why??? because the test data needs to be represented in the same space (11432 features)

In [58]:
X_test_tfidf.shape

(6251, 11432)

In [ ]:
#svm model?

In [59]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score

In [1]:
svm_model = SVC(kernel='linear')  # should I try another kernel? What does t depend on? 
#read about kernels here https://www.geeksforgeeks.org/svm-hyperparameter-tuning-using-gridsearchcv-ml/
svm_model.fit(X_train_tfidf, y_train)

NameError: name 'SVC' is not defined

In [61]:
y_predictions = svm_model.predict(X_test_tfidf)

In [ ]:
#evaluations

In [66]:
accuracy = round(accuracy_score(y_test, y_predictions), 2)
report = classification_report(y_test, y_predictions)

print(f"Accuracy: {accuracy}")
print("Classification Report:")
print(report)

Accuracy: 0.85
Classification Report:
              precision    recall  f1-score   support

           0       0.82      0.80      0.81      2472
           1       0.87      0.89      0.88      3779

    accuracy                           0.85      6251
   macro avg       0.85      0.84      0.84      6251
weighted avg       0.85      0.85      0.85      6251

